In [156]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    SystemMessagePromptTemplate,
    PromptTemplate
)
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_community.document_loaders.csv_loader import CSVLoader
import pandas as pd
import os

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [157]:


# Pastikan variabel lingkungan GROQ_API_KEY sudah ada
api_key = os.getenv("groq_api_key")
if api_key is None:
    raise ValueError("Did not find groq_api_key, please add an environment variable `GROQ_API_KEY` which contains it, or pass `groq_api_key` as a named parameter.")

In [158]:
# llm
llm = ChatGroq(
    model="llama3-8b-8192",
    api_key=api_key,
    temperature=0,
    model_kwargs={
        "top_p": 0.95,
        "frequency_penalty": 0.1,
        "presence_penalty": 0.2,
    },
    verbose=True
)

In [159]:
user_query = "What luxury hotels are available in Basel?"

## Step 1: Identify the intent of user's query

In [160]:
loader = CSVLoader(file_path="../data/few-shot/new_prompt_intents.csv")
doc_int = loader.load()

In [161]:
template = "You are a helpful assistant capable of detecting the intent behind a user's query."
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a helpful assistant capable of detecting the intent behind a user's query. Your job is to extract relevant intent from user queries based on patterns in documents.
    Respond with a JSON object. Provide no preamble or explanation.

    example:
    I'm booked on flight 30625 and my ticket number is 0060005435212386. Can you tell me my assigned seat?
    intent=CHECK_SEAT_ASSIGNMENT

    Documents:
    {document}

    Now, extract intents from the following user query:
    {user_query}

    <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["user_query", "document"],
)

detect_intent = prompt | llm | JsonOutputParser()
intent = detect_intent.invoke({"user_query": user_query, "document": doc_int})
print(intent)


{'intent': 'FIND_HOTEL_OPTIONS'}


## Step 2: Extract the entities from the user query
Load example prompt and completion pairs needed for entity recognition

In [162]:
loader = CSVLoader(file_path="../data/few-shot/new_prompt_ner.csv")
doc = loader.load()

In [163]:
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an AI assistant specializing in Named Entity Recognition (NER). Your task is to extract relevant entities from user queries based on patterns in documents. Follow these strict rules:

    1. Always respond with a single JSON object. Do not provide any explanation or additional text.
    2. Use lowercase keys in the JSON object.
    3. For dates, convert them to YYYY-MM-DD format.
    4. If no entities are found, return an empty JSON object:.

    Examples:

    example 1:
    user_query: Fetch the reservations from the second week of October 2023
    answer=Start Date: 2023-10-08, End Date: 2023-10-15
    
    example 2:
    user_query: I want to know the busiest departure months in 2023.
    answer=Year:'2023'

    Documents:
    {document}

    Now, extract entities from the following user query:
    {user_query}

    <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["user_query", "document"],
)

ner_detect = prompt | llm | JsonOutputParser()
ner = ner_detect.invoke({"user_query": user_query, "document": doc})
print(ner["entities"])


{'Location': 'Basel', 'Price Tier': 'Luxury', 'Attribute': 'Hotel Name'}


## Step 3: Table mapping

In [164]:
loader = CSVLoader(file_path="../data/few-shot/new_mapping.csv")
doc_map = loader.load()

In [165]:
template = "You are a helpful assistant capable of mapping detected intent to the correct list of SQLite tables."

In [166]:
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a helpful assistant capable of mapping detected intent to the correct list of SQLite tables.
    Provide NO preamble or explanation

    example:
    intent=RETRIEVE_RESERVATIONS
    answer=hotels|trip_recommendations|flights

    Documents:
    {document}

    User Intent:
    {intent}

    <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["intent", "document"],
)

detect_map = prompt | llm | StrOutputParser()
result_map = detect_map.invoke({"intent": intent["intent"], "document": doc_map})
print(result_map)


hotels


## Step 4: Load and filter table schemas

In [167]:
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///travel.sqlite")

In [168]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
context = toolkit.get_context()

print(context["table_info"])


CREATE TABLE aircrafts_data (
	aircraft_code TEXT, 
	model TEXT, 
	range INTEGER
)

/*
3 rows from aircrafts_data table:
aircraft_code	model	range
773	Boeing 777-300	11100
763	Boeing 767-300	7900
SU9	Sukhoi Superjet-100	3000
*/


CREATE TABLE airports_data (
	airport_code TEXT, 
	airport_name TEXT, 
	city TEXT, 
	coordinates TEXT, 
	timezone TEXT
)

/*
3 rows from airports_data table:
airport_code	airport_name	city	coordinates	timezone
ATL	Hartsfield-Jackson Atlanta International Airport	Atlanta	[33.6407, -84.4277]	America/New_York
PEK	Beijing Capital International Airport	Beijing	[40.0799, 116.6031]	Asia/Shanghai
DXB	Dubai International Airport	Dubai	[25.2532, 55.3657]	Asia/Dubai
*/


CREATE TABLE boarding_passes (
	ticket_no TEXT, 
	flight_id INTEGER, 
	boarding_no INTEGER, 
	seat_no TEXT
)

/*
3 rows from boarding_passes table:
ticket_no	flight_id	boarding_no	seat_no
0060005435212351	30625	1	2D
0060005435212386	30625	2	3G
0060005435212381	30625	3	4H
*/


CREATE TABLE bookings (
	bo

## Step 6: Text-to-SQL

In [169]:
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a SQL master expert capable of writing complex SQL query in SQLite.
    Please construct a SQL query using the information provided below:

    Input Parameters:
    -----------------
    INTENT: {intent}
    EXTRACTED_ENTITIES: {entities}
    MAPPED_TABLES: {tables}

    User Query:
    -----------
    {user_query}

    Table Schemas:
    --------------
    {table_schemas}

    <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["intent", "entities", "tables", "user_query", "table_schemas"],
)

sql_generate = prompt | llm | StrOutputParser()
sql_gen = sql_generate.invoke(
    {
        "intent": intent["intent"], 
        "entities": ner["entities"], 
        "tables": result_map, 
        "user_query": user_query, 
        "table_schemas": context["table_info"]
    }
)
print(sql_gen)


To answer the user query "What luxury hotels are available in Basel?" using the provided SQL tables, we can use the following SQL query:

```sql
SELECT * 
FROM hotels 
WHERE location = 'Basel' AND price_tier = 'Luxury' AND booked = 0;
```

This query will return all luxury hotels in Basel that are currently available (i.e., booked = 0).


In [171]:
QUERY = """
SELECT * 
FROM hotels 
WHERE location = 'Basel' AND price_tier = 'Luxury' AND booked = 0;
"""

# print(user_query)
db.run(QUERY)


"[(1, 'Hilton Basel', 'Basel', 'Luxury', '2024-04-22', '2024-04-20', 0)]"

## Step 7: Transform SQL results into a human friendly response